In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from utils import *

# 이슈 별, 초기 뜨거운 반응을 얻은 게시물 확인

In [8]:
event = "벤츠 화재"
communities = ['clien', 'bobae', 'fmkorea', 'naver_cafe']

In [9]:
per_community_dfs = []
for community in communities:
    posts_df = pd.read_csv(f'../data/{event}/{community}_posts.csv') 
    posts_df['from'] = community
    posts_df.created_at = posts_df.created_at.apply(parse_dates)

    comments_df = pd.read_csv(f'../data/{event}/{community}_comments.csv') 
    if community=='clien': #TODO: clien dataset cmt_author, post_id가 바뀌어 있음.
        comments_df.columns = ['cmt_author', 'cmt_count', 'post_id', 'cmt_created_at']
    comments_df = comments_df.groupby(['post_id'], as_index = False).agg({
        'cmt_author': ['count'],
    })

    comments_df.columns = comments_df.columns.droplevel(0)
    comments_df.columns = ['post_id', 'cmt_count']
    per_community_df = pd.merge(posts_df, comments_df, left_on='id', right_on='post_id', how='left')
    per_community_dfs.append(per_community_df)
df = pd.concat(per_community_dfs)

In [11]:
# Preprocessing
df = df.dropna(subset=['created_at']) # 생성 시간이 없는 게시물 제거
df.views = df.views.map(str).apply(remove_commna).apply(convert_str_to_int)
df.likes = df.likes.map(str).apply(remove_commna).apply(convert_str_to_float)
df.cmt_count = df.cmt_count.fillna(0).map(int)

# filtering by keyword & add created_day column
keywords = list(event.split())
filtered_df = filter_by_keyword(df, keywords)
filtered_df = add_created_day_col(filtered_df).sort_values(by=['created_at'])

In [12]:
filtered_df

,id,title,content,likes,url,author,views,created_at,updated_at,from,post_id,cmt_count,created_day
390,1523826,청라 전기차 화재영상이라네요...벤츠 eqe,아 남일같지 않네요 ㅠㅠ,1.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,면정학Iix3l인천,1768,2024-08-01 11:12:00,NaN,naver_cafe,1.523826e+06,17,2024-08-01
85,708549,인천 지하주차장에서 벤츠 전기차 화재...EQE? EQS? 둘 중 하나[6],https://x.com/_universe_sun_/status/1818816133...,0.0,https://www.bobaedream.co.kr//view?code=import...,투자의책임은본인,1149,2024-08-01 11:14:00,NaN,bobae,7.085490e+05,1,2024-08-01
389,1523836,청라 아파트 지하주차장 화재…벤츠 EQE 전기차 주차중 폭발 추정[단독영상],청라 아파트 지하주차장 화재…벤츠 EQE 전기차 주차중 폭발 추정[단독영상]인천 지...,1.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,톰I서울,22000,2024-08-01 11:18:00,NaN,naver_cafe,1.523836e+06,21,2024-08-01
388,1523849,진짜 이러면 전기차는 다 불안해야하나요...,"이번에 지하주차장 화재 차량은 EQE 벤츠차량 맞네요중국산, CATL, NCM622...",0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,뭐사야되나I없음I인천,1758,2024-08-01 11:26:00,NaN,naver_cafe,1.523849e+06,29,2024-08-01
84,7311560820,오늘 아침 인천 청라 전기차 화재 폭발 cctv.mp4,Video PlayerVideo 태그를 지원하지 않는 브라우저입니다.00:00Use...,7.0,https://www.fmkorea.com/7311560820,MOMO랜드연우,3326,2024-08-01 11:27:00,NaN,fmkorea,7.311561e+09,10,2024-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,1538817,벤츠EQE350+무상점검???,무상점검 세부 내용은 알 수 없죠 ???궁금하네요.화재가 밧데리 때문이라면 믿을 만...,0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,부드러운남EQE경주,591,2024-08-13 20:49:00,NaN,naver_cafe,1.538817e+06,12,2024-08-13
180,1538831,벤츠와 결별합니다,"앤트리 차량에서는 보조금을 받을수 있으니 SK on 배터리를 사용하고, 고가 차량은...",2.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,웅파이어 I i5 I 경기,1699,2024-08-13 21:01:00,NaN,naver_cafe,1.538831e+06,17,2024-08-13
179,1538867,이제는 완충을 못하니 주행가능거리도 줄겠지요,푸조 e-2008 입니다.벤츠 화재사건 발갱하기 며칠 전 찍었던건데요...350 k...,1.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,밀버그le2008l화성,383,2024-08-13 21:43:00,NaN,naver_cafe,1.538867e+06,2,2024-08-13
178,1538876,지하주차장 충전기 전원차단,벤츠 전기차 화재 후 저희 아파트 지하 주차장 충전기는 사용할 수 없게 전원이 안들...,0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,궁금해IC40I경기,572,2024-08-13 21:59:00,NaN,naver_cafe,1.538876e+06,18,2024-08-13
